In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
pd.set_option('display.max_columns', 100)

#### Обработаем тестовый датасет

In [3]:
df = pd.read_csv('course_project_test.csv')
# df.sample(5)

In [4]:
def mis_values_report(df):
    '''Функция считает и выводит пропущенные значения столбцов датафрейма'''
    mis_val = df.isnull().sum()
    perc_mis_val = 100 * df.isnull().sum() / len(df)
    report = pd.concat([mis_val, perc_mis_val], axis=1)
    report_columns = report.rename(columns = {0: 'Пропущенные значения', 1: '% от всех значений'})
    report_columns = report_columns[report_columns.iloc[:,1] != 0].sort_values('% от всех значений', ascending=False).round(1)
    print('Датафрейм имеет ' + str(df.shape[1]) + ' столбцов.\n' + str(report_columns.shape[0]) + ' столбцов имеют пропущенные значения')
    return report_columns

In [5]:
# mis_values_report(df)

In [6]:
df = df.drop(['Months since last delinquent','Number of Open Accounts','Maximum Open Credit','Current Credit Balance','Monthly Debt', 'Purpose'],axis=1)
# df.head()

In [7]:
# df['Bankruptcies'].value_counts().plot(kind='bar')
df['Bankruptcies'].fillna(float(0), inplace = True)
mis_values_report(df)

Датафрейм имеет 10 столбцов.
3 столбцов имеют пропущенные значения


,Пропущенные значения,% от всех значений
Annual Income,513,20.5
Credit Score,513,20.5
Years in current job,86,3.4


In [8]:
# plt.figure(figsize=(10,6))
# sns.countplot(df['Years in current job'])

In [9]:
df['Years in current job'].fillna('10+ years', inplace = True)
# mis_values_report(df)

In [10]:
# df['Credit Score'].hist()

In [11]:
df['Credit Score'] = df['Credit Score'].apply(lambda val: (val / 10) if val > 850 else val)
df['Credit Score'].fillna(df['Credit Score'].mean(), inplace=True)
# sns.boxplot(x=df['Credit Score'])

In [12]:
df['Annual Income'].fillna(df['Annual Income'].mean(), inplace=True)

In [13]:
# mis_values_report(df)

In [14]:
df['Term'].replace(('Short Term','Long Term'),(0,1), inplace=True)

In [15]:
df['Credit Score'] = df['Credit Score'].apply(lambda val: "Poor" if np.isreal(val) and val < 580 else val)
df['Credit Score'] = df['Credit Score'].apply(lambda val: "Average" if np.isreal(val) and (val >= 580 and val < 670) else val)
df['Credit Score'] = df['Credit Score'].apply(lambda val: "Good" if np.isreal(val) and (val >= 670 and val < 740) else val)
df['Credit Score'] = df['Credit Score'].apply(lambda val: "Very Good" if np.isreal(val) and (val >= 740 and val < 800) else val)
df['Credit Score'] = df['Credit Score'].apply(lambda val: "Exceptional" if np.isreal(val) and (val >= 800 and val <= 850) else val)

In [16]:
# sns.countplot(df['Credit Score'])

In [17]:
df = df.join(pd.get_dummies(df['Credit Score'], drop_first = True))
df.rename(index = str, columns={'Good':'Credit Good', 'Very Good':'Credit Very Good'})

,Home Ownership,Annual Income,Years in current job,Tax Liens,Years of Credit History,Number of Credit Problems,Bankruptcies,Term,Current Loan Amount,Credit Score,Credit Good,Credit Very Good
0,Rent,1.366922e+06,4 years,0.0,12.5,0.0,0.0,0,162470.0,Good,1,0
1,Rent,2.318380e+05,1 year,0.0,32.7,0.0,0.0,0,78298.0,Good,1,0
2,Home Mortgage,1.152540e+06,3 years,0.0,13.7,0.0,0.0,0,200178.0,Good,1,0
3,Home Mortgage,1.220313e+06,10+ years,0.0,17.0,0.0,0.0,0,217382.0,Good,1,0
4,Home Mortgage,2.340952e+06,6 years,0.0,23.6,0.0,0.0,1,777634.0,Good,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,Home Mortgage,1.020053e+06,10+ years,0.0,29.1,1.0,1.0,0,99999999.0,Very Good,0,1
2496,Home Mortgage,1.366922e+06,2 years,0.0,17.0,0.0,0.0,0,468512.0,Good,1,0
2497,Home Mortgage,1.171806e+06,2 years,0.0,12.8,0.0,0.0,0,430496.0,Good,1,0
2498,Rent,7.235200e+05,10+ years,0.0,28.8,0.0,0.0,0,257774.0,Very Good,0,1


In [18]:
df = df.drop(['Credit Score'], axis=1)

In [19]:
# df['Home Ownership'].value_counts().sort_values(ascending = True).plot(kind='bar', title='Number of credit based on Home ownership')

In [20]:
df = df.join(pd.get_dummies(df['Home Ownership'],drop_first = True))
df = df.drop(['Home Ownership'], axis=1)
# df.head()

In [21]:
df['Years in current job'] = df['Years in current job'].str.extract(r"(\d+)")
df['Years in current job'] = df['Years in current job'].astype(float)
df['Employment History'] = df['Years in current job'].apply(
    lambda x: 'Junior lvl' if x < 4 else ('Middle lvl' if x > 4 and x < 8 else 'Senior lvl'))
df = df.drop(['Years in current job'], axis=1)
# df.head()

In [22]:
df = df.join(pd.get_dummies(df['Employment History'],drop_first = True))

In [23]:
df = df.drop(['Employment History'], axis=1)
# df.head()

In [24]:
df['Credit Problems'] = df['Number of Credit Problems'].apply(
    lambda x: "No Credit Problems" if x==0 else ("Some Credit promblem" if x>0 and x<5 else "Big Credit Problems"))

In [25]:
df = df.join(pd.get_dummies(df['Credit Problems'],drop_first = True))
df = df.drop(['Credit Problems','Number of Credit Problems'], axis=1)
# df.head()

In [26]:
df['Credit Age'] = df['Years of Credit History'].apply(
    lambda x: "Short Credit Age" if x<5 else ("Good Credit Age" if x>5 and x<17 else "Excellent Credit Age"))

In [27]:
df = df.join(pd.get_dummies(df['Credit Age'],drop_first = True))
df = df.drop(['Credit Age','Years of Credit History'], axis =1)
# df.head()

In [28]:
df['Tax Liens'] = df['Tax Liens'].apply(
    lambda x: "No Tax Lien" if x==0 else ("Some Tax Liens" if x>0 and x<3 else "Many Tax Liens"))

In [29]:
df = df.join(pd.get_dummies(df['Tax Liens'],drop_first = True))
df = df.drop(['Tax Liens'],axis=1)
# df.head()

In [30]:
df['Bankruptcies'] = df['Bankruptcies'].apply(
    lambda x: "No Bankruptcies" if x==0 else ("Some Bankruptcies" if x>0 and x<3 else "Many Bankruptcies"))
df = df.join(pd.get_dummies(df['Bankruptcies'],drop_first = True))
df = df.drop(['Bankruptcies'],axis=1)
# df.head()

In [31]:
mean_income = df['Annual Income'].mean()
std_income = df['Annual Income'].std()
poorline = mean_income - std_income
richline = mean_income + std_income
print(poorline, richline)

549332.5668087583 2184510.7849778547


In [32]:
df['Annual Income'] = df['Annual Income'].apply(
    lambda x: "Low Income" if x <= poorline else ("Average Income" if x > poorline and x < richline else "High Income"))

In [33]:
df = df.join(pd.get_dummies(df['Annual Income'],drop_first = True))
df = df.drop(['Annual Income'], axis=1)
# df.head()

In [34]:
lmeanxoutlier = df[df['Current Loan Amount'] < 99999999.00 ]['Current Loan Amount'].mean()
lstddevxoutlier = df[df['Current Loan Amount'] < 99999999.00 ]['Current Loan Amount'].std()
lowrange = lmeanxoutlier - lstddevxoutlier
highrange = lmeanxoutlier + lstddevxoutlier
print(lowrange, highrange)

130222.3527286081 495560.1075528263


In [35]:
df['Current Loan Amount'] = df['Current Loan Amount'].apply(
    lambda x: "Small Loan" if x<=lowrange else ("Medium Loan" if x>lowrange and x<highrange else "Big Loan"))

In [36]:
df = df.join(pd.get_dummies(df['Current Loan Amount'],drop_first = True))
df = df.drop(['Current Loan Amount'], axis=1)
# df.head()

#### Загрузим модели

In [37]:
import joblib
catboost_upsamled = joblib.load('catboost_upsamled.pkl')
sgd_SMOTE = joblib.load('sgd_SMOTE.pkl')
xgbc_upsampled = joblib.load('xgbc_upsampled.pkl')

In [38]:
# catboost upsampled
y_pred_catboost = catboost_upsamled.predict(df)

# SGDClassifier SMOTE
y_pred_sgd = sgd_SMOTE.predict(df)

# Xg_Boost upsampled
y_pred_xgb = xgbc_upsampled.predict(df)

In [39]:
print(y_pred_catboost.shape)
print(y_pred_sgd.shape)
print(y_pred_xgb.shape)

(2500,)
(2500,)
(2500,)


In [40]:
num_id = np.arange(7500,10000)
num_id.shape

(2500,)

In [41]:
features = ['Id','Credit Default']
predict_catboost = pd.DataFrame({'Id': num_id, 'Credit Default': y_pred_catboost},
                       columns = features, dtype=np.int32)

In [42]:
predict_catboost.head()

,Id,Credit Default
0,7500,1
1,7501,1
2,7502,0
3,7503,0
4,7504,0


In [43]:
predict_sgd = pd.DataFrame({'Id': num_id, 'Credit Default': y_pred_sgd},
                       columns = features, dtype=np.int32)

In [44]:
predict_sgd.head()

,Id,Credit Default
0,7500,1
1,7501,1
2,7502,0
3,7503,1
4,7504,0


In [45]:
predict_xgb = pd.DataFrame({'Id': num_id, 'Credit Default': y_pred_xgb},
                       columns = features, dtype=np.int32)

In [46]:
predict_xgb.head()

,Id,Credit Default
0,7500,1
1,7501,1
2,7502,0
3,7503,0
4,7504,0


In [47]:
predict_catboost.to_csv('predict_catboost', index=None)
predict_sgd.to_csv('predict_sgd', index=None)
predict_xgb.to_csv('predict_xgb', index=None)